In [1]:
from sklearn.model_selection import train_test_split
import ast
# import transformers
# import numpy as np
# import os 
# import random
# import tokenizers
from transformers import AutoModelForCausalLM, AutoTokenizer, Qwen2Config, TextStreamer, TextIteratorStreamer, TrainingArguments
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
import os 
import datasets
from datasets import load_dataset, Dataset
import pandas as pd 
# import datasets
# from datasets import load_dataset, 
# import requests
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
from trl import SFTTrainer
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from sklearn.model_selection import train_test_split

/home/rpipc/anaconda3/envs/LLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
def clean_line(line):
        line = line.replace('\xa0', ' ')  
        line = line.replace('\u200c', '')  
        return line.strip()

persian_json_data = '/mnt/c/Users/Rpipc/Desktop/llama/farbod/f_d/datas/fa/davodi.csv'
persian_df = pd.read_csv(persian_json_data)
persian_df.drop_duplicates(inplace=True)
# persian_df['words'] = persian_df['words'] .apply(lambda x: clean_line(x))


prop_roles = set()
for _, row in persian_df.iterrows():
        prop_str = ast.literal_eval(row['propBankTags'])
        prop_roles.update(prop_str)
prop_roles = list(prop_roles)

frame_roles = set()
for _, row in persian_df.iterrows():
    framenet_str = ast.literal_eval(row['frameNetTags'])
    frame_roles.update(framenet_str)
frame_roles = list(frame_roles)

persian_df_tr, persian_df_temp = train_test_split(persian_df, random_state=42, test_size=.08)
persian_df_val, persian_df_te = train_test_split(persian_df_temp, random_state=42, test_size=.05)
del persian_df_temp
persian_df_tr.reset_index(drop=True, inplace=True)
persian_df_val.reset_index(drop=True, inplace=True)
persian_df_te.reset_index(drop=True, inplace=True)
# persian_df_te.to_csv(os.path.join(os.path.dirname(persian_json_data), 'persian_df_test.csv'), index=False)
print(persian_df_tr.shape)
print(persian_df_val.shape)
print(persian_df_te.shape)
persian_df_tr = Dataset.from_pandas(persian_df_tr)
persian_df_val = Dataset.from_pandas(persian_df_val)
persian_df_te = Dataset.from_pandas(persian_df_te)

(30090, 3)
(2486, 3)
(131, 3)


In [67]:
prop_roles

['B-RPT',
 'B-Arg2_B0',
 'B-NEG',
 'B-V',
 'B-ADV',
 'B-RCL1',
 'B-DIR',
 'B-A0-N-V',
 'B-ASP',
 'B-MNR',
 'B-PRP',
 'B-MOD',
 'B-T1',
 'B-A1',
 'B-ArgA',
 'B-PRD',
 'B-A1-N-V',
 'B-N-V',
 'B-ADJ',
 'B-DIS',
 'B-PUNC',
 'B-NVE-N-V',
 'B-CAU',
 'B-RCL2',
 'B-GOL',
 'B-A4',
 'B-COM',
 'B-A2-N-V',
 'B-LOC',
 'B-A3',
 'B-Def',
 'O',
 'B-CON',
 'B-EXT',
 'B-INS',
 'B-A0',
 'B-A2',
 'B-Top',
 'B-Arg2_B1',
 'B-TMP',
 'B-TMP-N-V',
 'B-REC',
 'B-NVE']

In [66]:
frame_roles

['رهبری_کنونی',
 'کنش',
 'حوزه_طبیعی',
 'وسیله_نیروی_محرکه',
 'وضعیت_جدید',
 'مشارکان',
 'موقعیت_بدن',
 'محل_هدف',
 'سختی',
 'جهت',
 'اعتیاد_آورنده',
 'وضعیت_نهایی',
 'جزءبدن',
 'طیف_وسیعی_از_گزینه',
 'مشارک_کانونی',
 'وسیله_حرکت',
 'تکرارها',
 'نقطه\u200cپایان',
 'شی',
 'درک\u200cکننده',
 'بازدیدکننده',
 'دارو',
 'جوهر',
 'الزام',
 'گیاه',
 'پیكره',
 'کارمند',
 'صدا',
 'بازدید\u200cکننده',
 'پرداخت_کننده',
 'مقصد',
 'موقعیت_مکانی',
 'مقدار_پیشرفت',
 'محل_نگهداری',
 'عملكرد',
 'رهبر_قدیمی',
 'سخنگو',
 'ذینفع',
 'اجزای_ساخت',
 'شرکت\u200cكننده_همراه',
 'مدرک_منفعل',
 'اشخاص_حقیقی',
 'حکم',
 'حركت_كننده_ارادی',
 'مضروب',
 'موقعیت_مکانی_متعارف',
 'فرآیند',
 'سمت_وفاداری',
 'قربانی',
 'نمونه_مورد_بحث',
 'قدیمی',
 'هم\u200cمشارک',
 'پول',
 'خصوصیت',
 'دانش',
 'محل_ثابت',
 'ارزیابی\u200c\u200cشونده',
 'وسیله\u200cی\u200cنقلیه',
 'موقعیت_برتر',
 'بدرفتارگر',
 'واسط',
 'دسته\u200cبندی',
 'تعداد',
 'زمان_مقصد',
 'شکل_مسیر',
 'برگزیده',
 'مقامات',
 'تخلف',
 'تم',
 'تناوب',
 'تجربه',
 'ناحیه_فرعی

In [23]:
persian_df.head()

,words,frameNetTags,propBankTags
0,خانم نزهت‌الدوله گرچه تا به حال سه تا شوهر کرد...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,خانم نزهت‌الدوله گرچه تا به حال سه تا شوهر کرد...,"['جستجو_کننده', 'جستجو_کننده', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,تو به من مجال دادی تا پروانه شوم .,"['مرجع_ذیصلاح', 'قهرمان', 'قهرمان', 'O', 'O', ...","['B-A0', 'B-A2', 'O', 'B-NVE', 'B-V', 'B-PRP',..."
3,تو به من مجال دادی تا پروانه شوم .,"['O', 'O', 'O', 'O', 'O', 'O', 'دسته_نهایی', '...","['O', 'O', 'O', 'O', 'O', 'O', 'B-A2', 'B-V', ..."
4,هر زمان این غریزه و فطرت و رشد سنی و عقلی و اج...,"['O', 'O', 'موجودیت', 'موجودیت', 'موجودیت', 'م...","['O', 'O', 'O', 'B-A2', 'O', 'O', 'O', 'O', 'O..."


In [7]:
persian_df.iloc[2,-1]

"['B-A0', 'B-A2', 'O', 'B-NVE', 'B-V', 'B-PRP', 'O', 'O', 'O']"

In [16]:


# Convert to list
list_representation = ast.literal_eval(persian_df.iloc[2,-1])
set(list_representation)

{'B-A0', 'B-A2', 'B-NVE', 'B-PRP', 'B-V', 'O'}

In [12]:
b = list()
b.extend([i for i in persian_df.iloc[2,-1]])
b

['[',
 "'",
 'B',
 '-',
 'A',
 '0',
 "'",
 ',',
 ' ',
 "'",
 'B',
 '-',
 'A',
 '2',
 "'",
 ',',
 ' ',
 "'",
 'O',
 "'",
 ',',
 ' ',
 "'",
 'B',
 '-',
 'N',
 'V',
 'E',
 "'",
 ',',
 ' ',
 "'",
 'B',
 '-',
 'V',
 "'",
 ',',
 ' ',
 "'",
 'B',
 '-',
 'P',
 'R',
 'P',
 "'",
 ',',
 ' ',
 "'",
 'O',
 "'",
 ',',
 ' ',
 "'",
 'O',
 "'",
 ',',
 ' ',
 "'",
 'O',
 "'",
 ']']

In [6]:
a = set()
a.update(persian_df.iloc[2,-1])
a

{' ',
 "'",
 ',',
 '-',
 '0',
 '2',
 'A',
 'B',
 'E',
 'N',
 'O',
 'P',
 'R',
 'V',
 '[',
 ']'}

In [36]:
set(clean_line(i) for i in frame_roles)

{'O',
 'آتش',
 'آزمایش',
 'آزمودنی',
 'آزمونگر',
 'آشپز',
 'آیتم',
 'ابتلا',
 'ابزار',
 'ابزار_بستن',
 'ابزار_حرارت',
 'ابزار_حرارت_دادن',
 'ابزارآلات',
 'ابعاد',
 'اتحاد',
 'اتهام',
 'اتهامات',
 'اثر',
 'اثر_سیستمیک',
 'اثر_هنری',
 'اثرپذیر',
 'اجارهدهنده',
 'اجارهکننده',
 'اجرا',
 'اجراکننده',
 'اجزا',
 'اجزاء',
 'اجزای_تشکیلدهنده',
 'اجزای_ساخت',
 'اجزای_سازنده',
 'احتمال',
 'احتمال_نماسازیشده',
 'اختلاف',
 'اداره_شونده',
 'ادراک_کننده_منفعل',
 'ادعای_اصلی',
 'ارائهدهنده',
 'ارائهدهنده_خدمات',
 'ارتباط',
 'ارتباط_برقرار_کننده',
 'ارتباطبرقرار_کننده',
 'ارتباطدهنده',
 'ارتباطگر',
 'ارجاع',
 'ارز_مبدا',
 'ارز_مقصد',
 'ارزش',
 'ارزش_دو',
 'ارزش_نهایی',
 'ارزش_یک',
 'ارزیاب',
 'ارزیابی',
 'ارزیابی_شونده',
 'ارزیابیشونده',
 'ارگانیسم',
 'استاندارد',
 'استنتاج',
 'استودیو',
 'اسناد',
 'اسیر',
 'اشتباه',
 'اشخاص_حقیقی',
 'اشغالکننده',
 'اشغالگر',
 'اصطلاح',
 'اصل',
 'اصل__الزام_آور',
 'اصل_الزام_آور',
 'اصل_و_نسب',
 'اصلاح',
 'اصلی',
 'اطلاعات',
 'اطلاعرسان',
 'اعتیاد_آورنده',
 'اعداد',
 '

In [53]:
str(frame_roles)

"{'رهبری_کنونی', 'کنش', 'حوزه_طبیعی', 'وسیله_نیروی_محرکه', 'وضعیت_جدید', 'مشارکان', 'موقعیت_بدن', 'محل_هدف', 'سختی', 'جهت', 'اعتیاد_آورنده', 'وضعیت_نهایی', 'جزءبدن', 'طیف_وسیعی_از_گزینه', 'مشارک_کانونی', 'وسیله_حرکت', 'تکرارها', 'نقطه\\u200cپایان', 'شی', 'درک\\u200cکننده', 'بازدیدکننده', 'دارو', 'جوهر', 'الزام', 'گیاه', 'پیكره', 'کارمند', 'صدا', 'بازدید\\u200cکننده', 'پرداخت_کننده', 'مقصد', 'موقعیت_مکانی', 'مقدار_پیشرفت', 'محل_نگهداری', 'عملكرد', 'رهبر_قدیمی', 'سخنگو', 'ذینفع', 'اجزای_ساخت', 'شرکت\\u200cكننده_همراه', 'مدرک_منفعل', 'اشخاص_حقیقی', 'حکم', 'حركت_كننده_ارادی', 'مضروب', 'موقعیت_مکانی_متعارف', 'فرآیند', 'سمت_وفاداری', 'قربانی', 'نمونه_مورد_بحث', 'قدیمی', 'هم\\u200cمشارک', 'پول', 'خصوصیت', 'دانش', 'محل_ثابت', 'ارزیابی\\u200c\\u200cشونده', 'وسیله\\u200cی\\u200cنقلیه', 'موقعیت_برتر', 'بدرفتارگر', 'واسط', 'دسته\\u200cبندی', 'تعداد', 'زمان_مقصد', 'شکل_مسیر', 'برگزیده', 'مقامات', 'تخلف', 'تم', 'تناوب', 'تجربه', 'ناحیه_فرعی', 'نورافكن', 'فرکانس', 'متخصص_پزشکی', 'بافت', 'الگو', 'تکرا

In [3]:
max_seq_length = 8192 
dtype = None 
load_in_4bit = True # 
model_path = '/mnt/c/Users/Rpipc/Desktop/llama/Llama-3.2-3B-Instruct-bnb-4bit'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path, 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
# to_save = '/home/data/Tavakoli/p_test/test_l/base_and_instruct_models/unsloth_Llama-3.2-3B-Instruct-bnb-4bit'
# if not os.apath.exists(to_save):
#     os.makedirs(to_save, exist_ok=True)
#     model.save_pretrained(to_save)
#     tokenizer.save_pretrained(to_save)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

==((====))==  Unsloth 2024.12.11: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.12.11 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
frame_prompt_template = """<|start_header_id|>system<|end_header_id|>
Please generate FrameNet roles for the provided text.
<|eot_id|><|start_header_id|>user<|end_header_id|>
You are an expert in the field of Semantic Role Labeling and lexical resources especially FrameNet.
You know anything about how to label sentence tokens with FrameNet frame elements.
Please use the following text:"+ {}+"Here are the frame elements you have to use for labeling:" + {}+"
Your task is to generate frame elements for the provided text.
The output should be a list of frame elements in a list format. IF the token do not have any frame element, put 'O'.
Make sure that you do NOT use any elements other than the ones I provided in this prompt.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{}
"""

prop_prompt_template = """<|start_header_id|>system<|end_header_id|>
Please generate PropBank roles for the provided text.
<|eot_id|><|start_header_id|>user<|end_header_id|>
You are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.
You know anything about how to label sentence tokens with PropBank roles.
Please use the following text:"+ {}+"Here are the propbank roles you have to use for labeling:" + {}+"
Your task is to generate PropBank roles for the provided text.
The output should be a list of roles in a list format. IF a token does not have any role, put 'O'.
Make sure that you do NOT use any roles other than the ones I provided in this prompt.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{}
"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_function(examples):
    words = examples["words"]
    frams = examples["frameNetTags"]
    props = examples["propBankTags"]
    texts = []

    # Generate FrameNet prompts
    for word, fram in zip(words, frams):
        text = frame_prompt_template.format(word, str(frame_roles), fram) + EOS_TOKEN
        
        texts.append(text)

    # Generate PropBank prompts
    for word, prop in zip(words, props):
        text = prop_prompt_template.format(word, str(prop_roles), prop) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}
main_dataset = persian_df_tr.map(formatting_prompts_function, remove_columns=persian_df_tr.column_names, batched=True,)
val_dataset = persian_df_val.map(formatting_prompts_function, remove_columns=persian_df_val.column_names, batched=True,) 
test_dataset = persian_df_te.map(formatting_prompts_function, remove_columns=persian_df_te.column_names, batched=True,) 
main_dataset = main_dataset.shuffle(seed=42)
val_dataset = val_dataset.shuffle(seed=42)
test_dataset = test_dataset.shuffle(seed=42)

# val_dataset = val_dataset.map(lambda examples: {
#     "text": examples["frame_text"] + examples["prop_text"]
# })

Map:   0%|          | 0/30090 [00:00<?, ? examples/s]

Map: 100%|██████████| 131/131 [00:00<00:00, 7020.25 examples/s]


In [108]:
test_dataset['text']

['<|start_header_id|>system<|end_header_id|>\nPlease generate PropBank roles for the provided text.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nYou are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.\nYou know anything about how to label sentence tokens with PropBank roles.\nPlease use the following text:"+ این ما هستیم که به انگلیس می\u200cگوییم ما را استعمار کن .+"Here are the propbank roles you have to use for labeling:" + [\'B-RPT\', \'B-Arg2_B0\', \'B-NEG\', \'B-V\', \'B-ADV\', \'B-RCL1\', \'B-DIR\', \'B-A0-N-V\', \'B-ASP\', \'B-MNR\', \'B-PRP\', \'B-MOD\', \'B-T1\', \'B-A1\', \'B-ArgA\', \'B-PRD\', \'B-A1-N-V\', \'B-N-V\', \'B-ADJ\', \'B-DIS\', \'B-PUNC\', \'B-NVE-N-V\', \'B-CAU\', \'B-RCL2\', \'B-GOL\', \'B-A4\', \'B-COM\', \'B-A2-N-V\', \'B-LOC\', \'B-A3\', \'B-Def\', \'O\', \'B-CON\', \'B-EXT\', \'B-INS\', \'B-A0\', \'B-A2\', \'B-Top\', \'B-Arg2_B1\', \'B-TMP\', \'B-TMP-N-V\', \'B-REC\', \'B-NVE\']+"\nYour task is to generat

In [60]:
persian_df_te['words']

['پدرم در یک کاروانسرا حجره\u200cای داشت و در آن خیاطی می\u200cکرد .',
 'کانت بیان می\u200cکند که داوری ذوقی از آن رو که بیان احساس می\u200cکند با معرفت عقلی و مفهومی تفاوت دارد .',
 'عیسی به عزلت از همۀ عالم کناره جست .',
 'پلیس نیویورک از زمان حملات یازدهم سپتامبر دو هزار و یک به صورت نظام\u200cمند از مساجد تجسس می\u200cکند .',
 'چهل روز زمان برد تا فاصلۀ میان آزادمرد بودن و دیندار بودن را طی کنیم .',
 'دیروز آمدیم شما را اسکورت بکنیم .',
 'دلاک دوباره سوزن را فروبرد پهلوان فریاد زد ، کدام اندام را می\u200cکشی ؟',
 'توجه کنید که هیچ ارتباط ارگانیک تاریخی ، فرهنگی و سیاسی بین چهار حوزۀ مهاجرتی که در ۴۰۰ سال نخست پس از اسلام در ایران نطفه بسته ، وجود ندارد .',
 'راه را باز کنید تا محققان و اندیشمندان بتوانند وارد شوند و توانمندی\u200cهای کشور را تجمیع کنید .',
 'پنجم اردیبهشت به راستی حکایت تکرار\u200cشده و تفسیر مجدد سورۀ فیل بود اما این بار اصحاب ابرهه نه با فیل بلکه با هلی\u200cکوپتر ، نه با هدف نابودی خانۀ خدا بلکه با هدف نابودی حکومت اسلام پای به عرصۀ تجاوز نهاد و خداوند نیز این ب

In [45]:
persian_df_te['frameNetTags']

["['مالک', 'O', 'O', 'O', 'مالکیت', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
 "['O', 'O', 'O', 'O', 'O', 'شیوه', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
 "['کنشگر', 'O', 'O', 'رویداد', 'رویداد', 'رویداد', 'O', 'O', 'O']",
 "['O', 'O', 'O', 'O', 'O', 'زمان', 'زمان', 'زمان', 'زمان', 'زمان', 'زمان', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
 "['شمارش', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
 "['زمان', 'زمان', 'مقصود', 'مقصود', 'مقصود', 'مقصود', 'O']",
 "['O', 'O', 'O', 'O', 'O', 'گوینده', 'O', 'O', 'O', 'پیام', 'پیام', 'پیام', 'پیام', 'O']",
 "['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'شمارش', 'O', 'زمان_نسبی', 'کل', 'کل', 'کل', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
 "['شی_مظروف', 'شی_مظروف', 'O', 'O', 'نتیجه', 'نتیجه', 'نتیجه', 'نتیجه', 'نتیجه', 'نتیجه', 'نتیجه', 'نتیجه', 'نتیجه', 'نتیجه', 'نتیجه', 'نتیجه', 'نتیجه', 'O']",
 "['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [44]:
persian_df_te['propBankTags']

["['B-A0', 'B-LOC', 'O', 'O', 'B-A1', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O']",
 "['O', 'O', 'O', 'O', 'B-N-V', 'B-MNR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
 "['B-A0', 'B-PRP', 'O', 'B-A1', 'O', 'O', 'B-NVE', 'B-V', 'O']",
 "['O', 'O', 'O', 'O', 'B-N-V', 'B-TMP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
 "['B-EXT', 'B-N-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
 "['B-TMP', 'B-V', 'O', 'O', 'O', 'B-PRP', 'O']",
 "['O', 'O', 'O', 'O', 'O', 'B-A0', 'B-NVE', 'O', 'O', 'O', 'O', 'O', 'B-A1', 'O']",
 "['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EXT', 'B-N-V', 'B-EXT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
 "['O', 'B-A1', 'B-NVE', 'B-V', 'B-PRP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
 "['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [87]:
import torch
torch.cuda.empty_cache()

In [19]:
main_dataset

Dataset({
    features: ['text'],
    num_rows: 60180
})

In [20]:
print(main_dataset["text"][40001])

<|start_header_id|>system<|end_header_id|>
Please generate FrameNet roles for the provided text.
<|eot_id|><|start_header_id|>user<|end_header_id|>
You are an expert in the field of Semantic Role Labeling and lexical resources especially FrameNet.
You know anything about how to label sentence tokens with FrameNet frame elements.
Please use the following text:"+ طی روزهای جاری دستورالعملی را به شبکۀ بانکی ابلاغ خواهیم_کرد تا هر کسی به هر میزان ارز بخواهد از بانک مرکزی دریافت کند .+"Here are the frame elements you have to use for labeling:" + ['ناشر', 'حسگر_1', 'ارائه\u200cدهنده_خدمات', 'وضعیت_توصیفی', 'كل', 'بخشی_از_بدن', 'پدیده_جستجو', 'بیان\u200cگر', 'اندازه', 'ارزش_یک', 'ارزیاب', 'اجزای_ساخت', 'میزبان', 'وضعیت_فعال', 'پشتیبان', 'موقعیت_نسبی', 'پدر', 'مشخصات_پایدار', 'اندیشه', 'دارو', 'آشپز', 'واگذارکننده', 'وسیله', 'کنش_اشغال', 'اجراکننده', 'رهبر_جدید', 'اقدام_پیشنهادی', 'قوه', 'علت_تشكیل', 'ابزار_حرارت_دادن', 'مجرم', 'قومیت', 'نابودگر', 'دقت', 'آیتم', 'عنوان', 'عمل_سومند', 'اهداکنند

In [7]:
val_dataset

Dataset({
    features: ['text'],
    num_rows: 4972
})

In [8]:
print(val_dataset["text"][1])

<|start_header_id|>system<|end_header_id|>
Please generate FrameNet roles for the provided text.
<|eot_id|><|start_header_id|>user<|end_header_id|>
You are an expert in the field of Semantic Role Labeling and lexical resources especially FrameNet.
You know anything about how to label sentence tokens with FrameNet frame elements.
Please use the following text:"+ پولاک به جای نام‌گذاری ، تابلوهایش را شماره‌گذاری می‌کرد چون معتقد بود اعداد کاملاً خنثی هستند .+"Here are the frame elements you have to use for labeling:" + ['علت_درونی', 'نورافكن', 'مقصود_تحمیلی', 'غذا', 'مرتکب_جنایت_یا_جنحه', 'اثر_سیستمیک', 'قدرت_یا_اشغالگر', 'نام_منبع', 'ارزش_دو', 'مالک_منطقه', 'مورد_استفاده', 'شرکت_کننده_1_1', 'افراد', 'شرکت\u200cكننده_همراه', 'منطقه_هدف', 'نشانه', 'مجموعه_اضافه_شده', 'فروبرنده', 'فرکانس', 'افروزنده', 'متصل_کننده', 'مشخصات', 'تولید_محصول', 'مقدار_مشخص', 'ارتباط\u200cدهنده', 'فرم', 'مدت_زمان', 'وسیله_جمع_آوری', 'دارو', 'مؤمن', 'هدف_تحمیلی', 'آتش', 'هدف_رویداد', 'پدیده_2', 'تولید_کننده_یا_ته

In [6]:
print(val_dataset["text"][1])

<|start_header_id|>system<|end_header_id|>
Please generate FrameNet roles for the provided text.
<|eot_id|><|start_header_id|>user<|end_header_id|>
You are an expert in the field of Semantic Role Labeling and lexical resources especially FrameNet.
You know anything about how to label sentence tokens with FrameNet frame elements.
Please use the following text:"+ پولاک به جای نام‌گذاری ، تابلوهایش را شماره‌گذاری می‌کرد چون معتقد بود اعداد کاملاً خنثی هستند .+"Here are the frame elements you have to use for labeling:" + ['ناشر', 'حسگر_1', 'ارائه\u200cدهنده_خدمات', 'وضعیت_توصیفی', 'كل', 'بخشی_از_بدن', 'پدیده_جستجو', 'بیان\u200cگر', 'اندازه', 'ارزش_یک', 'ارزیاب', 'اجزای_ساخت', 'میزبان', 'وضعیت_فعال', 'پشتیبان', 'موقعیت_نسبی', 'پدر', 'مشخصات_پایدار', 'اندیشه', 'دارو', 'آشپز', 'واگذارکننده', 'وسیله', 'کنش_اشغال', 'اجراکننده', 'رهبر_جدید', 'اقدام_پیشنهادی', 'قوه', 'علت_تشكیل', 'ابزار_حرارت_دادن', 'مجرم', 'قومیت', 'نابودگر', 'دقت', 'آیتم', 'عنوان', 'عمل_سومند', 'اهداکننده', 'مکان_استخدام', 'ارز

In [21]:
test_dataset['text'][0]

'<|start_header_id|>system<|end_header_id|>\nPlease generate PropBank roles for the provided text.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nYou are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.\nYou know anything about how to label sentence tokens with PropBank roles.\nPlease use the following text:"+ این ما هستیم که به انگلیس می\u200cگوییم ما را استعمار کن .+"Here are the propbank roles you have to use for labeling:" + [\'B-Def\', \'B-A0-N-V\', \'B-Arg2_B1\', \'B-EXT\', \'O\', \'B-PUNC\', \'B-ASP\', \'B-TMP-N-V\', \'B-A2\', \'B-PRP\', \'B-NVE\', \'B-DIR\', \'B-NVE-N-V\', \'B-CON\', \'B-COM\', \'B-DIS\', \'B-INS\', \'B-N-V\', \'B-V\', \'B-A1-N-V\', \'B-RCL2\', \'B-A1\', \'B-T1\', \'B-A3\', \'B-ADV\', \'B-MNR\', \'B-MOD\', \'B-ArgA\', \'B-RCL1\', \'B-Top\', \'B-NEG\', \'B-A0\', \'B-A2-N-V\', \'B-RPT\', \'B-A4\', \'B-LOC\', \'B-ADJ\', \'B-REC\', \'B-TMP\', \'B-Arg2_B0\', \'B-PRD\', \'B-GOL\', \'B-CAU\']+"\nYour task is to generate

In [5]:
main_dataset_selected_rows = main_dataset.select(range(2))
val_dataset_selected_rows = val_dataset.select(range(2))
test_dataset_selected_rows = test_dataset.select(range(50))

In [6]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = main_dataset_selected_rows,
    eval_dataset = val_dataset_selected_rows,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        # eval_steps= 1,
        eval_strategy='epoch',
        per_device_train_batch_size = 8,
        per_device_eval_batch_size= 8,
        gradient_accumulation_steps = 4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        # num_train_epochs = 32,
        max_steps = 1, 
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        # logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        # save_strategy="steps", 
        save_strategy="epoch", 
        logging_strategy = "epoch", 
        output_dir = '/mnt/c/Users/Rpipc/Desktop/llama/farbod/f_d/fine tuned models/unsloth/',
        # save_steps=132,
    ),
)


trainer.train()

Map (num_proc=2): 100%|██████████| 2/2 [00:01<00:00,  1.22 examples/s]
max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 1
 "-____-"     Number of trainable parameters = 24,313,856


Epoch,Training Loss,Validation Loss
1,2.500200,2.506797


TrainOutput(global_step=1, training_loss=2.500176429748535, metrics={'train_runtime': 24.0356, 'train_samples_per_second': 1.331, 'train_steps_per_second': 0.042, 'total_flos': 234620786429952.0, 'train_loss': 2.500176429748535, 'epoch': 1.0})

In [16]:
test_dataset_selected_rows["text"]

['<|start_header_id|>system<|end_header_id|>\nPlease generate PropBank roles for the provided text.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nYou are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.\nYou know anything about how to label sentence tokens with PropBank roles.\nPlease use the following text:"+ این ما هستیم که به انگلیس می\u200cگوییم ما را استعمار کن .+"Here are the propbank roles you have to use for labeling:" + [\'B-TMP\', \'B-CON\', \'B-REC\', \'B-ASP\', \'B-A1-N-V\', \'B-CAU\', \'B-TMP-N-V\', \'B-NVE\', \'B-PUNC\', \'B-LOC\', \'B-Def\', \'B-Arg2_B1\', \'B-A4\', \'B-RCL1\', \'B-A2\', \'O\', \'B-A1\', \'B-Arg2_B0\', \'B-NVE-N-V\', \'B-A3\', \'B-A0-N-V\', \'B-MNR\', \'B-A2-N-V\', \'B-N-V\', \'B-PRD\', \'B-INS\', \'B-ADJ\', \'B-NEG\', \'B-EXT\', \'B-V\', \'B-A0\', \'B-ADV\', \'B-DIS\', \'B-RCL2\', \'B-T1\', \'B-Top\', \'B-MOD\', \'B-PRP\', \'B-RPT\', \'B-COM\', \'B-GOL\', \'B-ArgA\', \'B-DIR\']+"\nYour task is to generat

In [67]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "/mnt/c/Users/Rpipc/Desktop/llama/Llama-3.2-3B-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_

In [68]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        # padding="max_length", 
        # padding="longest",
        # truncation=True, 
        # max_length=max_seq_length, 
        return_tensors="pt"  # Use PyTorch tensors to avoid later conversion issues
    )

# Assuming test_dataset_selected_rows is a Dataset object with a 'text' column
test_dataset_tokenized = test_dataset_selected_rows.map(tokenize_function, batched=True, remove_columns=['text'])
# test_dataset_tokenized = test_dataset_tokenized.remove_columns(['text'])
# trainer.args.per_device_eval_batch_size = 8
# test_dataset_tokenized = test_dataset_tokenized.to_dict()
# print(test_dataset_tokenized[0])
predictions = trainer.predict(test_dataset_tokenized)

if predictions.predictions.ndim == 3:  # (batch_size, seq_length, vocab_size)
    predicted_token_ids = predictions.predictions.argmax(axis=-1)
else:
    predicted_token_ids = predictions.predictions  # Already token IDs

decoded_inputs = tokenizer.batch_decode(test_dataset_tokenized["input_ids"], skip_special_tokens=True)
decoded_predictions = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)

pred_df = pd.DataFrame({
    "input": decoded_inputs,
    "prediction": decoded_predictions,
})

Map: 100%|██████████| 2/2 [00:00<00:00, 33.29 examples/s]


RuntimeError: CUDA driver error: out of memory

In [17]:
pred_df['prediction'][0]

'# (\n\n** provide aag\'s for the following sentence.Propimportassistant\n\nI can the AI in Prop of lingu Role Labeling ( Prop semantics. PropBank.I have that about Prop to generate roles using with PropBank roles?\n\n help Prop text asI (�یم ( در اینل می\u200cگویم + به بهحارگر. " ( is the tokensbank roles for can to label to the sentence\n\n "ARG-ARG\', \'B-g\',\',\', \'B-ARG0\',\',\', \'B-Arg\', \'B\'] \'O-ER\', \'O-ARG\', \'B-OP\',\',\', \'B-0\', \'B-ARGP\', \'B-P\', \'B-AT\', \'B-OM\',\',\', \'B-ARGJ \'B-OP\', \'B-IR\', \'B-ARG\', \'B-OM\', \'B-\', \'B-1\',\',\', \'B-P\',\', \'B-1\', \'B-MP\', \'B-0\', \'B-ARG\', \'B-OD\', \'B-OD\', \'B-ARG1\', \'B-CL1\', \'B-ARG\', \'B-ARG\', \'B-0\', \'B-1\',-V\', \'B-CL\', \'B-1\', \'B-ARG\', \'B-ARGJ\', \'B-ARG\', \'B-MP\', \'B-ARG1\',0\', \'B-ARGP\', \'B-ST\', \'B-ARGUS\',[\'These task is to label PropBank roles for the sentence text.Here text should be labeled list of tuples with the specific format. Here the token is not have a role assigned

In [12]:
predictions.predictions

array([[[ 1.7109375 ,  2.25      ,  6.375     , ..., -1.625     ,
         -1.625     , -1.625     ],
        [-9.1875    , -6.5       , -5.375     , ...,  9.8125    ,
          9.8125    ,  9.8125    ],
        [ 5.40625   ,  3.171875  ,  5.0625    , ..., -1.9765625 ,
         -1.9765625 , -1.9765625 ],
        ...,
        [ 0.27929688,  6.96875   ,  2.703125  , ...,  2.296875  ,
          2.296875  ,  2.296875  ],
        [ 0.29296875,  7.03125   ,  2.671875  , ...,  2.015625  ,
          2.015625  ,  2.015625  ],
        [ 0.69140625,  7.5       ,  2.828125  , ...,  1.5       ,
          1.5       ,  1.5       ]],

       [[ 1.7109375 ,  2.25      ,  6.375     , ..., -1.625     ,
         -1.625     , -1.625     ],
        [-9.1875    , -6.5       , -5.375     , ...,  9.8125    ,
          9.8125    ,  9.8125    ],
        [ 5.40625   ,  3.171875  ,  5.0625    , ..., -1.9765625 ,
         -1.9765625 , -1.9765625 ],
        ...,
        [ 3.03125   ,  1.6875    ,  3.671875  , ...,  

In [1]:
import torch
torch.cuda.empty_cache()

In [61]:
type(test_dataset_selected_rows)

datasets.arrow_dataset.Dataset

In [71]:
test_dataset_tokenized[0]

{'input_ids': [128000,
  128006,
  9125,
  128007,
  198,
  5618,
  7068,
  3998,
  26913,
  13073,
  369,
  279,
  3984,
  1495,
  627,
  128009,
  128006,
  882,
  128007,
  198,
  2675,
  527,
  459,
  6335,
  304,
  279,
  2115,
  315,
  75433,
  15766,
  9587,
  287,
  323,
  78686,
  5070,
  5423,
  3998,
  26913,
  627,
  2675,
  1440,
  4205,
  922,
  1268,
  311,
  2440,
  11914,
  11460,
  449,
  3998,
  26913,
  13073,
  627,
  5618,
  1005,
  279,
  2768,
  1495,
  29898,
  100408,
  101237,
  102483,
  100473,
  100360,
  82868,
  103514,
  118401,
  100318,
  103940,
  100633,
  100473,
  101237,
  100394,
  94253,
  103221,
  36138,
  100813,
  662,
  5283,
  8586,
  527,
  279,
  2047,
  17469,
  13073,
  499,
  617,
  311,
  1005,
  369,
  55402,
  3047,
  489,
  2570,
  33,
  9469,
  5901,
  518,
  364,
  33,
  12,
  5910,
  518,
  364,
  33,
  12,
  67713,
  518,
  364,
  33,
  12,
  66204,
  518,
  364,
  33,
  6830,
  16,
  11500,
  20198,
  518,
  364,
  33,
  12,

In [62]:
type(test_dataset_tokenized)

datasets.arrow_dataset.Dataset

In [56]:
test_dataset_tokenized[0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [118]:
persian_df.shape

(32707, 3)

In [43]:
persian_df1_prop = persian_df1[['words', 'propBankTags']]
persian_df1_frame = persian_df1[['words', 'frameNetTags']] 

In [119]:
persian_df_tr, persian_df_temp = train_test_split(persian_df, random_state=42, test_size=.08)
persian_df_val, persian_df_te = train_test_split(persian_df_temp, random_state=42, test_size=.05)
del persian_df_temp
persian_df_tr.reset_index(drop=True, inplace=True)
persian_df_te.reset_index(drop=True, inplace=True)

In [120]:
persian_df_tr.shape

(30090, 3)

In [121]:
persian_df_te.shape

(131, 3)

In [123]:
persian_df_val.shape

(2486, 3)

In [124]:
persian_df_tr.head()

,words,frameNetTags,propBankTags
0,ماه پس از ساعت 4 و 40 دقیقۀ بامداد و مریخ 30 د...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,در سیاست داخلی برقراری اصول مشروطیت و آزادی و ...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,از یک طرف انگلستان به اشرار پول می‌داد تا فضای...,"['O', 'O', 'O', 'اهداکننده', 'پذیرنده', 'پذیرن...","['B-ADV', 'O', 'O', 'B-A0', 'B-A2', 'O', 'B-A1..."
3,از شدت عطش با صدای بلند نعره می‌زند .,"['توضیح', 'توضیح', 'توضیح', 'شیوه', 'شیوه', 'ش...","['B-A1', 'O', 'O', 'B-MNR', 'O', 'O', 'B-NVE',..."
4,فردی یک میلیون بار دایرةالمعارف ویکی‌پدیا را و...,"['رویداد', 'تعداد_تکرار', 'تعداد_تکرار', 'O', ...","['O', 'O', 'B-EXT', 'B-N-V', 'O', 'O', 'O', 'O..."


In [61]:
persian_df_tr = pd.concat([persian_df1_prop_tr, persian_df1_frame_tr], ignore_index=True) 
persian_df_te = pd.concat([persian_df1_prop_te, persian_df1_frame_te], ignore_index=True)

In [63]:
persian_df_te

,words,propBankTags,frameNetTags
0,در حال نماز با دست خود یا با سر یا با ریش بازی...,"['B-TMP', 'O', 'O', 'B-A1', 'O', 'O', 'O', 'O'...",NaN
1,عرض کرد ، برادرم لباس‌های پشمینه می‌پوشد ؛ خان...,"['O', 'O', 'O', 'B-A0', 'O', 'O', 'O', 'O', 'O...",NaN
2,شب اول محرم یک ربع ( 15 دقیقه ) سینه زدیم و نو...,"['O', 'O', 'O', 'O', 'B-EXT', 'O', 'O', 'O', '...",NaN
3,گونه‌شناسی به مقایسۀ زبان‌ها می‌پردازد تا آنها...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-A1', 'B...",NaN
4,ابوسعید تا پایان عمر سیره و رفتار پیامبر اکرم ...,"['B-A0', 'B-TMP', 'O', 'O', 'O', 'O', 'O', 'O'...",NaN
...,...,...,...
651,جهان برای اولین بار از طریق « ISC » به فعالیت‌...,NaN,"['رویداد', 'O', 'ویژگی_مورد', 'O', 'رویداد', '..."
652,ما آن خطی هستیم که بیشترین تلاش را برای بالا ب...,NaN,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'هدف'..."
653,در طی عمل ، اضطراب و دلهره تمام وجودم را فراگر...,NaN,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
654,آیا اساساً کار هنر این نیست که جابجایی ایجاد ک...,NaN,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [45]:
persian_df1_prop.head()

,words,propBankTags
0,خانم نزهت‌الدوله گرچه تا به حال سه تا شوهر کرد...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,خانم نزهت‌الدوله گرچه تا به حال سه تا شوهر کرد...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,تو به من مجال دادی تا پروانه شوم .,"['B-A0', 'B-A2', 'O', 'B-NVE', 'B-V', 'B-PRP',..."
3,تو به من مجال دادی تا پروانه شوم .,"['O', 'O', 'O', 'O', 'O', 'O', 'B-A2', 'B-V', ..."
4,هر زمان این غریزه و فطرت و رشد سنی و عقلی و اج...,"['O', 'O', 'O', 'B-A2', 'O', 'O', 'O', 'O', 'O..."


In [132]:
persian_df1_prop['words'][15358:]

15358    برای این که کمی به استدلال شما استناد کرده_باش...
15359    برای اثبات این مطلب به دو آیۀ زیر استناد می‌کن...
15361    اهمیت این اقدام در آن است که برای اولین بار در...
15362                         هر چه دوست داری استنباط کن .
15363    آنها بیشتر دوست دارند حرف بزنند و دیگران به حر...
                               ...                        
32724    روسیه به زودی برای عمل به تعهدهای خود به اروپا...
32725    منازعه و مشاجرۀ پدر و مادر اقدام ناخوشایندی اس...
32726    منازعه و مشاجرۀ پدر و مادر اقدام ناخوشایندی اس...
32727    منازعه و مشاجرۀ پدر و مادر اقدام ناخوشایندی اس...
32728    منازعه و مشاجرۀ پدر و مادر اقدام ناخوشایندی اس...
Name: words, Length: 17349, dtype: object

In [109]:
persian_df.reset_index(drop=True, inplace=True)
lenn = [len(persian_df['words'][i]) for i in range(persian_df.shape[0])]
lenn2 = [len(persian_df['propBankTags'][i]) for i in range(persian_df.shape[0])]
lenn3 = [len(persian_df['frameNetTags'][i]) for i in range(persian_df.shape[0])]


In [110]:
print(lenn.index(max(lenn)))
print(lenn2.index(max(lenn2)))
print(lenn3.index(max(lenn3)))

5357
5360
9760


In [111]:
5357+5360+9760

20477

In [112]:
max(lenn)+max(lenn2)+max(lenn3)

3280

In [46]:
prop_roles = set()
for _, row in persian_df1.iterrows():
        prop_str = row['propBankTags']
        prop_roles.update(prop_str)


frame_roles = set()
for _, row in persian_df1.iterrows():
    framenet_str = row['frameNetTags']
    frame_roles.update(framenet_str)

In [23]:
messages = [
    {"role": "system", "content": "How can I help you?"},
    {"role": "user", "content": "You are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.\
You know anything about how to label sentence tokens with PropBank roles.\
Please use the following text:"+ persian_df['words'][0]+"Here are the propbank roles you have to use for labeling:\
Your task is to generate PropBank roles for the provided text.\
The output should be a list of roles in a list format. IF a token does not have any role, put 'O'.\
Make sure that you do NOT use any roles other than the ones I provided in this prompt."},
{"role": "assistant", "content":""}
]

In [121]:
prompt_template = """<|start_header_id|>system<|end_header_id|>
Please generate PropBank roles for the provided text.
<|eot_id|><|start_header_id|>user<|end_header_id|>
You are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.
You know anything about how to label sentence tokens with PropBank roles.
Please use the following text:"+ {}+"Here are the propbank roles you have to use for labeling:" + {}+"
Your task is to generate PropBank roles for the provided text.
The output should be a list of roles in a list format. IF a token does not have any role, put 'O'.
Make sure that you do NOT use any roles other than the ones I provided in this prompt.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{}
"""

In [48]:
prompt_template.format(persian_df1['words'][0], str(prop_roles), persian_df1['propBankTags'][0])

'<|start_header_id|>system<|end_header_id|>\nPlease generate PropBank roles for the provided text.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nYou are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.\nYou know anything about how to label sentence tokens with PropBank roles.\nPlease use the following text:"+ خانم نزهت\u200cالدوله گرچه تا به حال سه تا شوهر کرده و شش بار زاییده و دو تا از دخترهایش هم به خانۀ داماد فرستاده_شده\u200cاند و حالا دیگر برای خودش مادربزرگ شده_است ، باز هم عقیده دارد که پیری و جوانی دست خود آدم است و گرچه سر و همسر و خویشان و دوستان می\u200cگویند که پنجاه سالی دارد ولی هنوز دو دستی به جوانی\u200cاش چسبیده و هنوز هم در جستجوی شوهر ایده\u200cآل به این در و آن در می\u200cزند .+"Here are the propbank roles you have to use for labeling:" + {\'E\', \'D\', \'r\', \'T\', \'p\', \'J\', \'M\', \'O\', \'G\', \'A\', \'f\', \'C\', \'2\', \'P\', \'N\', \'0\', \' \', \'-\', \'e\', \'4\', \'L\', \'S\', \'g\', \'3\', \'B\', \'I\

In [22]:
messages

[{'role': 'system', 'content': 'How can I help you?'},
 {'role': 'user',
  'content': 'You are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.You know anything about how to label sentence tokens with PropBank roles.Please use the following text:خانم نزهت\u200cالدوله گرچه تا به حال سه تا شوهر کرده و شش بار زاییده و دو تا از دخترهایش هم به خانۀ داماد فرستاده_شده\u200cاند و حالا دیگر برای خودش مادربزرگ شده_است ، باز هم عقیده دارد که پیری و جوانی دست خود آدم است و گرچه سر و همسر و خویشان و دوستان می\u200cگویند که پنجاه سالی دارد ولی هنوز دو دستی به جوانی\u200cاش چسبیده و هنوز هم در جستجوی شوهر ایده\u200cآل به این در و آن در می\u200cزند .Here are the propbank roles you have to use for labeling:{\'E\', \'D\', \'r\', \'T\', \'p\', \'J\', \'M\', \'O\', \'G\', \'A\', \'f\', \'C\', \'2\', \'P\', \'N\', \'0\', \' \', \'-\', \'e\', \'4\', \'L\', \'S\', \'g\', \'3\', \'B\', \'I\', \'X\', "\'", \'[\', \'1\', \'R\', \']\', \'o\', \'V\', \'_\', \',\', \'U\'}

In [24]:
  messages = [{'role': 'system', 'content': 'How can I help you?'},
 {'role': 'user',
  'content': 'You are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.You know anything about how to label sentence tokens with PropBank roles.Please use the following text:خانم نزهت\u200cالدوله گرچه تا به حال سه تا شوهر کرده و شش بار زاییده و دو تا از دخترهایش هم به خانۀ داماد فرستاده_شده\u200cاند و حالا دیگر برای خودش مادربزرگ شده_است ، باز هم عقیده دارد که پیری و جوانی دست خود آدم است و گرچه سر و همسر و خویشان و دوستان می\u200cگویند که پنجاه سالی دارد ولی هنوز دو دستی به جوانی\u200cاش چسبیده و هنوز هم در جستجوی شوهر ایده\u200cآل به این در و آن در می\u200cزند .Here are the propbank roles you have to use for labeling:{\'E\', \'D\', \'r\', \'T\', \'p\', \'J\', \'M\', \'O\', \'G\', \'A\', \'f\', \'C\', \'2\', \'P\', \'N\', \'0\', \' \', \'-\', \'e\', \'4\', \'L\', \'S\', \'g\', \'3\', \'B\', \'I\', \'X\', "\'", \'[\', \'1\', \'R\', \']\', \'o\', \'V\', \'_\', \',\', \'U\'}Your task is to generate PropBank roles for the provided text.The output should be a list of roles in a list format. IF a token does not have any role, put \'O\'.Make sure that you do NOT use any roles other than the ones I provided in this prompt.'}]


In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Qwen2Config, TextStreamer, TextIteratorStreamer, TrainingArguments

/home/rpipc/anaconda3/envs/LLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [119]:
pth = '/mnt/c/Users/Rpipc/Desktop/llama/Llama-3.2-1B-Instruct-bnb-4bit'
tok = AutoTokenizer.from_pretrained(pth)
# model = AutoModelForCausalLM.from_pretrained(pth)

In [117]:
tok.chat_template

'{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- if strftime_now is defined %}\n        {%- set date_string = strftime_now("%d %b %Y") %}\n    {%- else %}\n        {%- set date_string = "26 Jul 2024" %}\n    {%- endif %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{{- "Cutting

In [26]:
tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 22 Dec 2024\n\nHow can I help you?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYou are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.You know anything about how to label sentence tokens with PropBank roles.Please use the following text:خانم نزهت\u200cالدوله گرچه تا به حال سه تا شوهر کرده و شش بار زاییده و دو تا از دخترهایش هم به خانۀ داماد فرستاده_شده\u200cاند و حالا دیگر برای خودش مادربزرگ شده_است ، باز هم عقیده دارد که پیری و جوانی دست خود آدم است و گرچه سر و همسر و خویشان و دوستان می\u200cگویند که پنجاه سالی دارد ولی هنوز دو دستی به جوانی\u200cاش چسبیده و هنوز هم در جستجوی شوهر ایده\u200cآل به این در و آن در می\u200cزند .Here are the propbank roles you have to use for labeling:{\'E\', \'D\', \'r\', \'T\', \'p\', \'J\', \'M\', \'O\', \'G\', \'A\', \'f\', \'C\', \'2\', \'P\', \'N\', \'0\', \' \', \'-\', \'e\', \'4\', \

In [27]:
test_dataset['text'][0]

'<|start_header_id|>system<|end_header_id|>\nPlease generate PropBank roles for the provided text.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nYou are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.\nYou know anything about how to label sentence tokens with PropBank roles.\nPlease use the following text:"+ این ما هستیم که به انگلیس می\u200cگوییم ما را استعمار کن .+"Here are the propbank roles you have to use for labeling:" + [\'B-Def\', \'B-A0-N-V\', \'B-Arg2_B1\', \'B-EXT\', \'O\', \'B-PUNC\', \'B-ASP\', \'B-TMP-N-V\', \'B-A2\', \'B-PRP\', \'B-NVE\', \'B-DIR\', \'B-NVE-N-V\', \'B-CON\', \'B-COM\', \'B-DIS\', \'B-INS\', \'B-N-V\', \'B-V\', \'B-A1-N-V\', \'B-RCL2\', \'B-A1\', \'B-T1\', \'B-A3\', \'B-ADV\', \'B-MNR\', \'B-MOD\', \'B-ArgA\', \'B-RCL1\', \'B-Top\', \'B-NEG\', \'B-A0\', \'B-A2-N-V\', \'B-RPT\', \'B-A4\', \'B-LOC\', \'B-ADJ\', \'B-REC\', \'B-TMP\', \'B-Arg2_B0\', \'B-PRD\', \'B-GOL\', \'B-CAU\']+"\nYour task is to generate

In [ ]:
test_dataset['text'][0].split("<|start_header_id|>assistant<|end_header_id|>\n")[1].split('\n<|eot_id|>')

['<|start_header_id|>system<|end_header_id|>\nPlease generate PropBank roles for the provided text.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nYou are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.\nYou know anything about how to label sentence tokens with PropBank roles.\nPlease use the following text:"+ این ما هستیم که به انگلیس می\u200cگوییم ما را استعمار کن .+"Here are the propbank roles you have to use for labeling:" + [\'B-Def\', \'B-A0-N-V\', \'B-Arg2_B1\', \'B-EXT\', \'O\', \'B-PUNC\', \'B-ASP\', \'B-TMP-N-V\', \'B-A2\', \'B-PRP\', \'B-NVE\', \'B-DIR\', \'B-NVE-N-V\', \'B-CON\', \'B-COM\', \'B-DIS\', \'B-INS\', \'B-N-V\', \'B-V\', \'B-A1-N-V\', \'B-RCL2\', \'B-A1\', \'B-T1\', \'B-A3\', \'B-ADV\', \'B-MNR\', \'B-MOD\', \'B-ArgA\', \'B-RCL1\', \'B-Top\', \'B-NEG\', \'B-A0\', \'B-A2-N-V\', \'B-RPT\', \'B-A4\', \'B-LOC\', \'B-ADJ\', \'B-REC\', \'B-TMP\', \'B-Arg2_B0\', \'B-PRD\', \'B-GOL\', \'B-CAU\']+"\nYour task is to generat

In [7]:
def split_chat_template(example):
    if "<|start_header_id|>assistant<|end_header_id|>\n" in example["text"]:
        parts = example["text"].split("<|start_header_id|>assistant<|end_header_id|>\n")
        return {"input": parts[0].strip(), "label": parts[1].split('\n<|eot_id|>')[0].strip()}
    else:
        return {"input": example["text"], "label": ""}  # Handle cases without labels
        
processed_dataset = test_dataset_selected_rows.map(split_chat_template, remove_columns=['text'])
print(processed_dataset[0])

Map: 100%|██████████| 50/50 [00:00<00:00, 4744.69 examples/s]

{'input': '<|start_header_id|>system<|end_header_id|>\nPlease generate PropBank roles for the provided text.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nYou are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.\nYou know anything about how to label sentence tokens with PropBank roles.\nPlease use the following text:"+ این ما هستیم که به انگلیس می\u200cگوییم ما را استعمار کن .+"Here are the propbank roles you have to use for labeling:" + [\'B-A3\', \'B-NVE-N-V\', \'B-TMP\', \'B-A1\', \'B-MOD\', \'B-ArgA\', \'B-A0\', \'B-NVE\', \'B-TMP-N-V\', \'B-CAU\', \'B-PRD\', \'B-MNR\', \'B-LOC\', \'B-ADJ\', \'B-DIS\', \'B-Arg2_B1\', \'B-CON\', \'B-A2\', \'B-DIR\', \'B-A2-N-V\', \'B-V\', \'B-PUNC\', \'B-ASP\', \'B-GOL\', \'B-RPT\', \'B-PRP\', \'B-A1-N-V\', \'B-A4\', \'B-RCL2\', \'B-N-V\', \'B-T1\', \'B-ADV\', \'B-A0-N-V\', \'B-Top\', \'B-COM\', \'B-INS\', \'B-EXT\', \'O\', \'B-RCL1\', \'B-REC\', \'B-Arg2_B0\', \'B-NEG\', \'B-Def\']+"\nYour task is t

In [40]:
processed_dataset

Dataset({
    features: ['input', 'label'],
    num_rows: 2
})

In [10]:
def tokenize_chat_data(example):
    # Tokenize the input
    inputs = tokenizer(
        example["input"], 
        # padding="longest", 
        padding=True,
        truncation=True, 
        # max_length=128
    )

    # Tokenize the label
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["label"], 
            # padding="longest",
            padding=True,
            truncation=True, 
            # max_length=128
        )

    # Add tokenized labels
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = processed_dataset.map(tokenize_chat_data, batched=True, remove_columns=['input', 'label'])
print(tokenized_dataset[0])

Map: 100%|██████████| 50/50 [00:00<00:00, 242.37 examples/s]

{'input_ids': [128000, 128006, 9125, 128007, 198, 5618, 7068, 3998, 26913, 13073, 369, 279, 3984, 1495, 627, 128009, 128006, 882, 128007, 198, 2675, 527, 459, 6335, 304, 279, 2115, 315, 75433, 15766, 9587, 287, 323, 78686, 5070, 5423, 3998, 26913, 627, 2675, 1440, 4205, 922, 1268, 311, 2440, 11914, 11460, 449, 3998, 26913, 13073, 627, 5618, 1005, 279, 2768, 1495, 29898, 100408, 101237, 102483, 100473, 100360, 82868, 103514, 118401, 100318, 103940, 100633, 100473, 101237, 100394, 94253, 103221, 36138, 100813, 662, 5283, 8586, 527, 279, 2047, 17469, 13073, 499, 617, 311, 1005, 369, 55402, 3047, 489, 2570, 33, 6830, 18, 518, 364, 33, 11500, 4592, 11500, 20198, 518, 364, 33, 9469, 5901, 518, 364, 33, 6830, 16, 518, 364, 33, 5364, 2114, 518, 364, 33, 12, 2803, 32, 518, 364, 33, 6830, 15, 518, 364, 33, 11500, 4592, 518, 364, 33, 9469, 5901, 11500, 20198, 518, 364, 33, 12, 5158, 52, 518, 364, 33, 12, 6616, 35, 518, 364, 33, 5364, 27416, 518, 364, 33, 12, 12321, 518, 364, 33, 12, 1846, 41, 518

In [13]:
tokenized_dataset[0]

{'input_ids': [128000,
  128006,
  9125,
  128007,
  198,
  5618,
  7068,
  3998,
  26913,
  13073,
  369,
  279,
  3984,
  1495,
  627,
  128009,
  128006,
  882,
  128007,
  198,
  2675,
  527,
  459,
  6335,
  304,
  279,
  2115,
  315,
  75433,
  15766,
  9587,
  287,
  323,
  78686,
  5070,
  5423,
  3998,
  26913,
  627,
  2675,
  1440,
  4205,
  922,
  1268,
  311,
  2440,
  11914,
  11460,
  449,
  3998,
  26913,
  13073,
  627,
  5618,
  1005,
  279,
  2768,
  1495,
  29898,
  100408,
  101237,
  102483,
  100473,
  100360,
  82868,
  103514,
  118401,
  100318,
  103940,
  100633,
  100473,
  101237,
  100394,
  94253,
  103221,
  36138,
  100813,
  662,
  5283,
  8586,
  527,
  279,
  2047,
  17469,
  13073,
  499,
  617,
  311,
  1005,
  369,
  55402,
  3047,
  489,
  2570,
  33,
  12,
  6616,
  47,
  518,
  364,
  33,
  6830,
  18,
  518,
  364,
  33,
  6830,
  16,
  518,
  364,
  33,
  9607,
  1669,
  518,
  364,
  33,
  6830,
  19,
  518,
  364,
  33,
  5364,
  2114,
  5

In [12]:
trainer.args.per_device_eval_batch_size= 1
predictions = trainer.predict(tokenized_dataset)
import torch.nn.functional as F
import torch
from evaluate import load

losses = []
for logits, labels in zip(predictions.predictions, predictions.label_ids):
        logits_tensor = torch.tensor(logits)
        labels_tensor = torch.tensor(labels)
        loss = F.cross_entropy(logits_tensor.view(-1, logits_tensor.size(-1)), labels_tensor.view(-1), reduction="none")
        loss_per_sample = loss.view(labels_tensor.size(0), -1).mean(dim=1)  # Mean over tokens
        losses.append(loss_per_sample.mean().item())
# Decode inputs, predictions, and labels
decoded_inputs = tokenizer.batch_decode(tokenized_dataset["input_ids"], skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(tokenized_dataset["labels"], skip_special_tokens=True)
decoded_predictions = tokenizer.batch_decode(predictions.predictions.argmax(axis=-1), skip_special_tokens=True)

def compute_exact_match(predictions, references):
    matches = [
        pred.strip() == ref.strip()
        for pred, ref in zip(predictions, references)
    ]
    accuracy = sum(matches) / len(matches) * 100
    return accuracy

exact_match_accuracy = compute_exact_match(decoded_predictions, decoded_labels)

# Compute BLEU score
bleu = load("bleu")
bleu_score = bleu.compute(
    predictions=[pred.strip() for pred in decoded_predictions],
    references=[[ref.strip()] for ref in decoded_labels]
)["bleu"]

# Compute ROUGE score
rouge = load("rouge")
rouge_score = rouge.compute(
    predictions=[pred.strip() for pred in decoded_predictions],
    references=[ref.strip() for ref in decoded_labels]
)["rougeL"]

# Compute token-level accuracy
def compute_token_accuracy(predicted_ids, label_ids):
    total_tokens = 0
    correct_tokens = 0

    for pred, ref in zip(predicted_ids, label_ids):
        for p_token, r_token in zip(pred, ref):
            if r_token != -100:  # Ignore padding tokens in labels
                total_tokens += 1
                if p_token == r_token:
                    correct_tokens += 1

    return (correct_tokens / total_tokens) * 100 if total_tokens > 0 else 0.0

predicted_token_ids = predictions.predictions.argmax(axis=-1)
token_accuracy = compute_token_accuracy(predicted_token_ids, predictions.label_ids)

# Save metrics and predictions
results_path = "/mnt/c/Users/Rpipc/Desktop/llama/farbod/f_d/fine tuned models/unsloth/test_predictions_with_metrics.csv"
metrics_path = "/mnt/c/Users/Rpipc/Desktop/llama/farbod/f_d/fine tuned models/unsloth/metrics_summary.txt"

# Save predictions with individual losses
pred_df = pd.DataFrame({
    "input": decoded_inputs,
    "label": decoded_labels,
    "prediction": decoded_predictions,
    "loss": losses,  # Add token-level losses
})
pred_df.to_csv(results_path, index=False)

# Save overall metrics
with open(metrics_path, "w") as f:
    f.write(f"Exact Match Accuracy: {exact_match_accuracy:.2f}%\n")
    f.write(f"Token-Level Accuracy: {token_accuracy:.2f}%\n")
    f.write(f"BLEU Score: {bleu_score:.4f}\n")
    # f.write(f"ROUGE-L Score: {rouge_score:.4f}\n")

print(f"Results saved to {results_path}")
print(f"Metrics summary saved to {metrics_path}")

TypeError: Unsupported types (<class 'unsloth.models._utils.EmptyLogits'>) passed to `_pad_across_processes`. Only nested list/tuple/dicts of objects that are valid for `is_torch_tensor` should be passed.

In [53]:
results_df['input'][0]

'system\nPlease generate PropBank roles for the provided text.\nuser\nYou are an expert in the field of Semantic Role Labeling and lexical resources especially PropBank.\nYou know anything about how to label sentence tokens with PropBank roles.\nPlease use the following text:"+ این ما هستیم که به انگلیس می\u200cگوییم ما را استعمار کن.+"Here are the propbank roles you have to use for labeling:" + [\'B-Def\', \'B-A0-N-V\', \'B-Arg2_B1\', \'B-EXT\', \'O\', \'B-PUNC\', \'B-ASP\', \'B-TMP-N-V\', \'B-A2\', \'B-PRP\', \'B-NVE\', \'B-DIR\', \'B-NVE-N-V\', \'B-CON\', \'B-COM\', \'B-DIS\', \'B-INS\', \'B-N-V\', \'B-V\', \'B-A1-N-V\', \'B-RCL2\', \'B-A1\', \'B-T1\', \'B-A3\', \'B-ADV\', \'B-MNR\', \'B-MOD\', \'B-ArgA\', \'B-RCL1\', \'B-Top\', \'B-NEG\', \'B-A0\', \'B-A2-N-V\', \'B-RPT\', \'B-A4\', \'B-LOC\', \'B-ADJ\', \'B-REC\', \'B-TMP\', \'B-Arg2_B0\', \'B-PRD\', \'B-GOL\', \'B-CAU\']+"\nYour task is to generate PropBank roles for the provided text.\nThe output should be a list of roles in a l

In [ ]:
test_dataset_tokenized[0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [123]:
len(tok.encode(prompt_template.format(persian_df['words'][5357], str(prop_roles), persian_df['propBankTags'][5360]))) + 1 

1263

In [131]:
# long_len_tr = [len(tok.encode(main_dataset['text'][i])) for i in range(main_dataset.num_rows)]
# long_len_val = [len(tok.encode(val_dataset['text'][i])) for i in range(val_dataset.num_rows)]
# long_len_test = [len(tok.encode(test_dataset['text'][i])) for i in range(test_dataset.num_rows)]
def compute_length(batch):
    # Tokenize the batch and compute lengths
    return {'length': [len(tok.encode(text)) for text in batch['text']]}

# Apply the map function to each dataset
long_len_tr = main_dataset.map(compute_length, batched=True)
long_len_val = val_dataset.map(compute_length, batched=True)
long_len_test = test_dataset.map(compute_length, batched=True)

# Extract lengths
lengths_tr = long_len_tr['length']
lengths_val = long_len_val['length']
lengths_test = long_len_test['length']

# Get the maximum lengths
max_length_tr = max(lengths_tr)
max_length_val = max(lengths_val)
max_length_test = max(lengths_test)

# Print the maximum lengths
print(f"Max length for training dataset: {max_length_tr}")
print(f"Max length for validation dataset: {max_length_val}")
print(f"Max length for test dataset: {max_length_test}")

Map: 100%|██████████| 262/262 [00:01<00:00, 136.43 examples/s]

Max length for training dataset: 7688
Max length for validation dataset: 7539
Max length for test dataset: 7189


In [129]:
print(max(long_len_tr))
print(max(long_len_val))
print(max(long_len_test))

7189